<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>

# 1.0 모델 내보내기
이 노트북에서는 PyTorch를 사용해 트레이닝된 BERT 체크포인트를 NVIDIA Triton Inference Server로 내보내는 옵션을 살펴보겠습니다.

**[1.1 개요: 최적화 및 성능](#1.1-개요:-최적화-및-성능)<br>**
**[1.2 BERT 체크포인트 내보내기](#1.2-BERT-체크포인트-내보내기)<br>**
&nbsp; &nbsp; &nbsp; &nbsp; [1.2.1 Triton 모델 리포지토리](#1.2.1-Triton-모델-리포지토리)<br>
&nbsp; &nbsp; &nbsp; &nbsp; [1.2.2 TorchScript 내보내기](#1.2.2-TorchScript-내보내기)<br>
**[1.3 내보내기 테스트](#1.3-내보내기-테스트)<br>**
**[1.4 TorchScript 외](#1.4-TorchScript-외)<br>**
&nbsp; &nbsp; &nbsp; &nbsp; [1.4.1 연습: TensorRT 최적화 활성화](#1.4.1-연습:-TensorRT-최적화-활성화)<br>
**[1.5 성능 비교](#1.5-성능-비교)<br>**

# 1.1 개요: 최적화 및 성능
트레이닝된 모델 최적화는 대역폭 및 지연 시간으로 측정되는 추론 성능에 매우 큰 영향을 미칩니다. 프로젝트 요구 사항이 지식 증류 또는 프루닝과 같은 고급 기법에 엔지니어링 노력을 투자하는 것을 정당화하지는 않지만 모델 최적화 도구를 사용하여 상당한 모델 성능 개선을 달성할 수 있습니다. 아래 다이어그램은 최적화되지 않은 TensorFlow를 사용하여 배포된 모델, TensorRT로 사후 처리된 동일한 모델, 그리고 TensorRT로 완전히 최적화된 모델 간의 추론 성능 차이를 보여줍니다. 

<img src="images/TFvTRT.jpg" alt="헤더" style="width: 600px;"/>

최신 추론 서버는 일반적으로 둘 이상의 모델 형식을 실질적으로 지원하여 더 광범위한 프로젝트, 도구 및 환경 설정을 충족합니다. 이 수업에서는 PyTorch를 사용해 트레이닝된 BERT 체크포인트를 사용하고 Triton Inference Server에 배포하므로 PyTorch 기반 모델을 배포하는 옵션을 중점적으로 살펴보겠습니다. 이러한 옵션에는 다음이 포함됩니다.
   - PyTorch JIT/TorchScript
   - ONNX 런타임
   - ONNX-TensorRT
   - TensorRT
    
Triton Server가 다음을 포함해 훨씬 더 폭넓은 배포 메커니즘 세트를 지원한다는 점에 주목해야 합니다.
   - TensorFlow GraphDef
   - TensorFlow 저장 모델
   - Caffe 2 내보내기
   - 맞춤형 모델(어떤 맞춤형 실행 파일도 될 수 있음)

이 섹션에서는 위에 열거된 배포 엔진 중 일부를 사용해 모델을 배포하는 방법과 각 옵션이 성능에 미치는 영향을 살펴보겠습니다. 또한 몇몇 주요 설정 즉, 배치 크기와 숫자 정밀도(FP32 및 FP16)를 실험해 보겠습니다.

# 1.2 BERT 체크포인트 내보내기

배포할 BERT 모델 체크포인트인 <code>bert_qa.pt</code>는 `data` 디렉토리에 위치해 있어야 합니다. 

In [1]:
!ls data/*.pt

data/bert_qa.pt


이 파일은 BERT-Large 네트워크의 표준 체크포인트로, [SQuAD(Stanford Question Answering Dataset)](https://arxiv.org/abs/1606.05250)에서 파인튜닝(Fine-tuning)되었습니다. 

#### 헬퍼 스크립트
우리는 다양한 배포 구성을 살펴보면서 일부 단계를 여러 번 반복할 것입니다.  따라서 구성 설정 및 결과에 주의를 집중할 수 있도록 몇 가지 헬퍼 스크립트를 사용하여 프로세스를 부분적으로 자동화할 것입니다.  관심이 있다면 코드 세부 정보를 직접 살펴볼 수 있습니다.

- [utilities/wait_for_triton_server.sh](utilities/wait_for_triton_server.sh): API를 통해 Triton Server의 "라이브" 및 "준비" 상태를 확인합니다.
- [deployer/deployer.py](deployer/deployer.py): 체크포인트를 배포 가능한 모델로 변환해서 내보냅니다.
- [utilities/run_perf_analyzer_local.sh](utilities/run_perf_analyzer_local.sh): [perf_analyzer](https://github.com/triton-inference-server/client/blob/main/src/c++/perf_analyzer/README.md) 애플리케이션을 사용해 성능을 측정합니다.
- [utilities/run_warmup.sh](utilities/run_warmup.sh): `perf_analyzer`를 사용하여 몇 가지 추론을 실행하여 모델을 웜업합니다. 모델을 웜업하면 더 안정적인 측정값을 얻을 수 있습니다.

Triton Server가 컨테이너에 배포되었으며 포트 "8000"의 호스트 "triton"에서 사용할 수 있습니다. 다음 셀을 실행하여 API에서 "200 OK" HTTP 응답을 확인합니다.

In [2]:
# Set the server hostname and check it - you should get a message that "Triton Server is ready!"
tritonServerHostName = "triton"
!./utilities/wait_for_triton_server.sh {tritonServerHostName}

Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!


## 1.2.1 Triton 모델 리포지토리
Triton Server가 시작되면 일반적으로 이 서버는 모델이 호스팅되는 로컬 또는 원격 파일 시스템을 관찰하도록 구성됩니다. 관찰되는 디렉토리를 *모델 리포지토리*라고 합니다. Triton Server를 시작하는 일반적인 명령은 옵션을 사용하여 모델 리포지토리의 위치를 식별합니다.<br>
```bash
tritonserver --model-repository="/path/to/model/repository"
```

모델 리포지토리는 다음 레이아웃을 사용해야 합니다.

```python
<model-repository-path>/
  <model-name>/
    [config.pbtxt]
    [<output-labels-file> ...]
    <version>/
      <model-definition-file>
    <version>/
      <model-definition-file>
    ...
  <model-name>/
    [config.pbtxt]
    [<output-labels-file> ...]
    <version>/
      <model-definition-file>
    <version>/
      <model-definition-file>
    ...
  ...
```

이 랩 컨테이너는 <code>./model_repository</code> 폴더를 모델 리포지토리로 사용하도록 구성되어 있으므로 이 폴더 내의 모든 변경 사항이 Triton Server의 동작에 영향을 미칩니다.<br/>

새로운 모델을 Triton에 노출하려면 다음을 수행해야 합니다. <br/>
   1. 모델 리포지토리에 새 모델 폴더를 만듭니다. 폴더의 이름은 사용자/애플리케이션에 노출할 서비스의 이름을 반영해야 합니다.<br/>
   2. 모델 폴더 내에서 해당 모델의 기본 서빙 구성을 포함하는 <code>config.pbtxt</code> 파일을 만듭니다.<br/>
   3. 또한 모델 폴더 내에서 모델의 복사본을 포함하는 폴더를 적어도 하나 이상 만듭니다. 폴더의 이름은 모델의 버전 이름을 반영합니다. 동일한 모델의 여러 버전을 만들고 호스팅할 수 있습니다.<br/>
    
다음으로, 모델을 Triton으로 내보내는 프로세스를 살펴보겠습니다.

## 1.2.2 TorchScript 내보내기

랩의 이 부분에서는 다음 작업을 수행합니다.
   - PyTorch 체크포인트를 [TorchScript](https://pytorch.org/docs/stable/jit.html#torchscript)로 변환
   - Triton 구성 파일 생성
   - 생성된 자산을 모델 리포지토리에 배포
아래 셀을 실행하십시오. PyTorch 체크포인트를 로드하고 TorchScript로 변환하는 데에는 1~2분 정도 소요될 수 있습니다.

In [3]:
modelName = "bertQA-torchscript"

In [4]:
!python ./deployer/deployer.py \
    --ts-script \
    --save-dir ./candidatemodels \
    --triton-model-name {modelName} \
    --triton-model-version 1 \
    --triton-max-batch-size 8 \
    --triton-dyn-batching-delay 0 \
    --triton-engine-count 1 \
    -- --checkpoint "/dli/task/data/bert_qa.pt" \
    --config_file ./bert_config.json \
    --vocab_file ./vocab \
    --predict_file ./squad/v1.1/dev-v1.1.json \
    --do_lower_case \
    --batch_size=8 

deploying model bertQA-torchscript in format pytorch_libtorch
/usr/local/lib/python3.10/dist-packages/torch/jit/_recursive.py:260: UserWarning: 'bias' was found in ScriptModule constants,  but it is a non-constant parameter. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "

conversion correctness test results
-----------------------------------
maximal absolute error over dataset (L_inf):  0.02516031265258789

average L_inf error over output tensors:  0.013570129871368408
variance of L_inf error over output tensors:  0.00011137690313015962
stddev of L_inf error over output tensors:  0.010553525625598283

time of error check of native model:  0.7686853408813477 seconds
time of error check of ts model:  1.4161465167999268 seconds

done


`deployer.py` 스크립트가 `bert_qa.pt` 체크포인트를 로드하고 이를 `ts-script` 형식으로 `bertQA-torchscript`라는 폴더에 배포한 다음 버전 `1`로 표시합니다. 더 고급에 해당하는 설정은 나중에 설명하겠습니다. 지금은 스크립트에 의해 생성된 파일을 확인해보겠습니다.

In [5]:
!ls -al ./candidatemodels/bertQA-torchscript/
!ls -al ./candidatemodels/bertQA-torchscript/1

total 16
drwxr-xr-x 3 root root 4096 Mar  4 05:55 .
drwxr-xr-x 3 root root 4096 Mar  4 05:55 ..
drwxr-xr-x 2 root root 4096 Mar  4 05:55 1
-rw-r--r-- 1 root root  568 Mar  4 05:55 config.pbtxt
total 1309300
drwxr-xr-x 2 root root       4096 Mar  4 05:55 .
drwxr-xr-x 3 root root       4096 Mar  4 05:55 ..
-rw-r--r-- 1 root root 1340709950 Mar  4 05:55 model.pt


예상한 대로, 스크립트가 모델을 TorchScript 형식으로 내보낸 다음 `model.pt`로 저장했습니다. 또한 `config.pbtxt` 파일을 생성했습니다. <br> 
한번 살펴보겠습니다.

In [6]:
!cat ./candidatemodels/bertQA-torchscript/config.pbtxt

name: "bertQA-torchscript"
platform: "pytorch_libtorch"
max_batch_size: 8
input [
{
    name: "input__0"
    data_type: TYPE_INT64
    dims: [384]
},
{
    name: "input__1"
    data_type: TYPE_INT64
    dims: [384]
},
{
    name: "input__2"
    data_type: TYPE_INT64
    dims: [384]
}
]
output [
{
    name: "output__0"
    data_type: TYPE_FP32
    dims: [384]
}, 
{
    name: "output__1"
    data_type: TYPE_FP32
    dims: [384]
}
]
optimization {
  cuda {
    graphs: 0
  }
}
instance_group [
    {
        count: 1
        kind: KIND_GPU
        gpus: [ 0 ]
    }
]

구성 파일은 매우 단순하며 다음을 정의합니다.
   - 모델의 이름
   - 추론에 사용할 플랫폼의 유형, 이 경우에는 `pytorch_libtorch`
   - 네트워크에서 사용되는 입력 및 출력 차원
   - 사용된 최적화, 이 경우에는 GPU 및 기본 TorchScript 최적화 
   - 인스턴스 그룹 구성, 이 경우에는 인스턴스 그룹 수가 1로 설정되며, 이는 모델 복사본 한 개만 GPU 메모리(GPU 0이 사용됨)에 저장됨을 의미합니다.
    
모델을 배포하려면 폴더를 Triton 모델 리포지토리로 이동합니다.

In [7]:
!mv ./candidatemodels/bertQA-torchscript model_repository/

축하합니다!  첫 번째 모델을 Triton Inference Server에 성공적으로 배포했습니다!

나중에 다시 돌아와서 자세한 구성에 대해 이야기하겠지만 지금은 모델이 수행되는 방식을 살펴보겠습니다.

#  1.3 내보내기 테스트
아래 셀을 실행하여 추론 프로세스를 시작하고 추론 성능을 간단하게 측정합니다. 먼저, 일부 구성을 설정하겠습니다. `maxConcurrency`는 2로 설정되며, 이는 스트레스 테스트가 두 번 실행됨을 의미합니다. 첫 번째 실행에서는 단일 스레드만 사용하고 두 번째 실행에서는 2개의 스레드를 사용하여 서버를 쿼리합니다. 동시 모델 실행 또는 동적 배치 기능을 켜지 않으면 서버를 쿼리하는 두 개의 프로세스 실행 성능에 어떤 영향을 미칠거라고 생각하십니까?<br/>
- 대역폭이 증가할까요, 감소할까요?<br/>
- 지연 시간이 증가할까요, 감소할까요?<br/>

In [8]:
modelVersion = "1"
precision = "fp32"
batchSize = "8"
maxLatency = "500"
maxClientThreads = "10"
maxConcurrency = "2"
dockerBridge = "host"
resultsFolderName = "1"
profilingData = "utilities/profiling_data_int64"
measurement_request_count = 50
percentile_stability = 85
stability_percentage = 50

In [9]:
%%time
modelName = "bertQA-torchscript"
maxConcurrency = "2"
batchSize = "8"
print("Running: "+modelName)
!./utilities/run_perf_analyzer_local.sh \
                    {modelName} \
                    {modelVersion} \
                    {precision} \
                    {batchSize} \
                    {maxLatency} \
                    {maxClientThreads} \
                    {maxConcurrency} \
                    {tritonServerHostName} \
                    {dockerBridge} \
                    {resultsFolderName} \
                    {profilingData} \
                    {measurement_request_count} \
                    {percentile_stability} \
                    {stability_percentage}

Running: bertQA-torchscript
Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!
*** Measurement Settings ***
  Batch size: 8
  Service Kind: Triton
  Using "count_windows" mode for stabilization
  Minimum number of samples in each window: 50
  Latency limit: 500 msec
  Concurrency limit: 2 concurrent requests
  Using synchronous calls for inference
  Stabilizing using p85 latency

Request concurrency: 1
  Pass [1] throughput: 35.33 infer/sec. p85 latency: 141283 usec
  Pass [2] throughput: 57.1356 infer/sec. p85 latency: 141775 usec
  Pass [3] throughput: 55.9936 infer/sec. p85 latency: 142262 usec
  Pass [4] throughput: 55.9924 infer/sec. p85 latency: 142523 usec
  Client: 
    Request count: 162
    Throughput: 56.3407 infer/sec
    Avg client overhead: 0.00%
    Avg send request rate: 7.05 infer/sec
    p50 latency: 141768 usec
    p85 latency: 142262 usec
    p90 latency: 142482 usec
    p95 latency: 142672 usec
    p99 latency: 142784 usec
    Avg g

모든 것이 정상일 경우 다음 예제 결과와 비슷한 출력이 표시되어 두 가지 구성 전반에서의 추론 성능을 보여주어야 합니다.<br/>
<img src="images/InferenceJob1.png" alt="추론 작업 1의 예제 출력" style="width: 1200px;"/>

"오류: 모델 메타데이터를 가져오지 못했습니다."라는 메시지가 표시될 경우 셀을 다시 실행해 보십시오.


# 1.4 TorchScript 외

모델을 Triton에 배포하는 다른 경로인 <a href="https://onnx.ai">ONNX(Open Neural Network Exchange)</a>를 살펴보겠습니다. ONNX는 뉴럴 네트워크 모델을 표현하고 교환하는 오픈 형식입니다. ONNX는 공통 모델을 구축하는 데 사용되는 공통 연산자 세트뿐 아니라 그러한 모델을 교환하기 위한 파일 형식을 정의합니다. ONNX의 이점은 비교적 광범위하게 채택되며 딥러닝 프레임워크 또는 배포 도구와 같은 <a href="https://onnx.ai/supported-tools.html">다양한 딥러닝 도구</a> 간에 모델을 교환하는 데 사용될 수 있다는 점입니다. 여기에는 ONNX 모델을 사용할 수 있는 TensorRT도 포함됩니다. </br>

앞서와 마찬가지로 모델을 내보내는 것으로 시작하지만 이번에는 ONNX 형식을 사용합니다. 앞서 사용한 내보내기 도구를 활용하되, 내보내기 형식을 <code>ts-script</code>에서 <code>onnx</code>로 변경합니다.

In [10]:
modelName = "bertQA-onnx"
exportFormat = "onnx"

In [11]:
!python ./deployer/deployer.py \
    --{exportFormat} \
    --save-dir ./candidatemodels \
    --triton-model-name {modelName} \
    --triton-model-version 1 \
    --triton-max-batch-size 8 \
    --triton-dyn-batching-delay 0 \
    --triton-engine-count 1 \
    -- --checkpoint ./data/bert_qa.pt \
    --config_file ./bert_config.json \
    --vocab_file ./vocab \
    --predict_file ./squad/v1.1/dev-v1.1.json \
    --do_lower_case \
    --batch_size=8

deploying model bertQA-onnx in format onnxruntime_onnx
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__2
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic a

<a href="https://github.com/pytorch/pytorch/blob/master/torch/csrc/jit/docs/serialization.md">TorchScript 직렬화 형식</a>과 마찬가지로, <a href="https://onnx.ai/get-started.html">ONNX 형식</a>은 매우 쉽게 확인할 수 있습니다(형식 부분을 사람이 판독할 수 있음). 내보내기에서 생성한 자산을 살펴보겠습니다.

In [12]:
!ls -al ./candidatemodels/bertQA-onnx/
!ls -al ./candidatemodels/bertQA-onnx/1

total 16
drwxr-xr-x 3 root root 4096 Mar  4 06:03 .
drwxr-xr-x 3 root root 4096 Mar  4 06:03 ..
drwxr-xr-x 2 root root 4096 Mar  4 06:03 1
-rw-r--r-- 1 root root  561 Mar  4 06:03 config.pbtxt
total 1305596
drwxr-xr-x 2 root root       4096 Mar  4 06:03 .
drwxr-xr-x 3 root root       4096 Mar  4 06:03 ..
-rw-r--r-- 1 root root 1336922015 Mar  4 06:03 model.onnx


다시 한번 말하지만 모델뿐 아니라 구성 파일도 있으며 이번에는 ONNX 형식으로 저장되었습니다. 

Triton에서 ONNX 기반 내보내기를 실행할 수 있는 몇 가지 옵션이 있습니다.
- ONNX 런타임을 활용할 수 있음 </br>
- 대신 사용할 TensorRT 엔진을 생성하기 위해 TensorRT에 ONNX 자산 구문 분석을 요청할 수 있음 </br>

두 접근 방식을 모두 시도해 보고 이것이 추론 성능에 미치는 영향을 살펴보겠습니다. 현재 ONNX 모델을 배포하기 위해 모델을 모델 리포지토리로 이동합니다...

In [13]:
!mv ./candidatemodels/bertQA-onnx model_repository/

...그런 다음 10가지 동시 실행 수준에서 스트레스 테스트 코드를 실행합니다.

In [14]:
%%time
modelName = "bertQA-onnx"
maxConcurrency = "10"
batchSize = "8"
print("Running: "+modelName)

!./utilities/run_perf_analyzer_local.sh \
                    {modelName} \
                    {modelVersion} \
                    {precision} \
                    {batchSize} \
                    {maxLatency} \
                    {maxClientThreads} \
                    {maxConcurrency} \
                    {tritonServerHostName} \
                    {dockerBridge} \
                    {resultsFolderName} \
                    {profilingData} \
                    {measurement_request_count} \
                    {percentile_stability} \
                    {stability_percentage}

Running: bertQA-onnx
Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!
*** Measurement Settings ***
  Batch size: 8
  Service Kind: Triton
  Using "count_windows" mode for stabilization
  Minimum number of samples in each window: 50
  Latency limit: 500 msec
  Concurrency limit: 10 concurrent requests
  Using synchronous calls for inference
  Stabilizing using p85 latency

Request concurrency: 1
  Pass [1] throughput: 67.9906 infer/sec. p85 latency: 117405 usec
  Pass [2] throughput: 67.9894 infer/sec. p85 latency: 117387 usec
  Pass [3] throughput: 67.989 infer/sec. p85 latency: 117356 usec
  Client: 
    Request count: 153
    Throughput: 67.9897 infer/sec
    Avg client overhead: 0.00%
    Avg send request rate: 8.55 infer/sec
    p50 latency: 117306 usec
    p85 latency: 117387 usec
    p90 latency: 117407 usec
    p95 latency: 117440 usec
    p99 latency: 117880 usec
    Avg gRPC time: 117315 usec (marshal 4 usec + response wait 117310 usec + unma

결과를 살펴보십시오. 10개의 모든 동시 실행 수준에서 벤치마크를 실행했습니까, 또는 벤치마크가 조기에 시간 초과되었습니까? 우리가 구성한 500ms 시간 제한과 관련해서 요청 지연 시간은 어떻게 되었습니까?</br>

이제 TensorRT 실행에 적합하게 구성할 수 있도록 ONNX 모델을 다시 살펴보겠습니다.</br>

In [15]:
modelName = "bertQA-onnx-trt-fp16"
exportFormat = "onnx"

In [16]:
!python ./deployer/deployer.py \
    --{exportFormat} \
    --save-dir ./candidatemodels \
    --triton-model-name {modelName} \
    --triton-model-version 1 \
    --triton-max-batch-size 8 \
    --triton-dyn-batching-delay 0 \
    --triton-engine-count 1 \
    -- --checkpoint ./data/bert_qa.pt \
    --config_file ./bert_config.json \
    --vocab_file ./vocab \
    --predict_file ./squad/v1.1/dev-v1.1.json \
    --do_lower_case \
    --batch_size=8

deploying model bertQA-onnx-trt-fp16 in format onnxruntime_onnx
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__2
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each 

다시 한번 말하지만 위의 명령으로 구성 파일뿐 아니라 ONNX 내보내기까지 생성되어야 합니다. 

In [17]:
!ls -al ./candidatemodels/bertQA-onnx-trt-fp16/

total 16
drwxr-xr-x 3 root root 4096 Mar  4 06:06 .
drwxr-xr-x 3 root root 4096 Mar  4 06:05 ..
drwxr-xr-x 2 root root 4096 Mar  4 06:06 1
-rw-r--r-- 1 root root  570 Mar  4 06:06 config.pbtxt


## 1.4.1 연습: TensorRT 최적화 활성화

TensorRT를 활성화하려면 "config.pbtxt" 구성 파일에 추가 섹션을 추가해야 합니다. 특히, <code>최적화</code> 섹션에 추가 세그먼트를 추가해야 합니다.

```text
optimization {
   execution_accelerators {
      gpu_execution_accelerator : [ {
         name : "tensorrt"
         parameters { key: "precision_mode" value: "FP16" }
      }]
   }
cuda { graphs: 0 }
}
```

#### 연습 단계:
1. TensorRT를 활성화하도록 [config.pbtxt](candidatemodels/bertQA-onnx-trt-fp16/config.pbtxt)를 수정합니다. 필요에 따라 언제든지 [솔루션](solutions/ex-1-4-1_config.pbtxt)을 살펴보십시오.
2. 변경 사항을 저장한 후(주 메뉴: 파일 -> 파일 저장), 아래 셀을 사용하여 폴더를 모델 리포지토리로 이동합니다. 

In [ ]:
# quick fix!
!cp solutions/ex-1-4-1_config.pbtxt candidatemodels/bertQA-onnx-trt-fp16/config.pbtxt

In [18]:
!mv ./candidatemodels/bertQA-onnx-trt-fp16 model_repository/

3. 다음 셀에서 프로파일링 도구를 실행하여 성능에 미치는 영향을 확인합니다. 서버가 모델을 TensorRT로 마이그레이션할 때까지 기다려야 하므로 이 작업을 시작하려면 시간이 다소 소요될 수 있습니다.

In [19]:
%%time
modelName = "bertQA-onnx-trt-fp16"
maxConcurrency = "10"
batchSize = "8"
print("Running: " + modelName)

!./utilities/run_perf_analyzer_local.sh \
                    {modelName} \
                    {modelVersion} \
                    {precision} \
                    {batchSize} \
                    {maxLatency} \
                    {maxClientThreads} \
                    {maxConcurrency} \
                    {tritonServerHostName} \
                    {dockerBridge} \
                    {resultsFolderName} \
                    {profilingData} \
                    {measurement_request_count} \
                    {percentile_stability} \
                    {stability_percentage}

Running: bertQA-onnx-trt-fp16
Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!
*** Measurement Settings ***
  Batch size: 8
  Service Kind: Triton
  Using "count_windows" mode for stabilization
  Minimum number of samples in each window: 50
  Latency limit: 500 msec
  Concurrency limit: 10 concurrent requests
  Using synchronous calls for inference
  Stabilizing using p85 latency

Request concurrency: 1
  Pass [1] throughput: 67.9921 infer/sec. p85 latency: 117531 usec
  Pass [2] throughput: 67.9883 infer/sec. p85 latency: 117447 usec
  Pass [3] throughput: 67.989 infer/sec. p85 latency: 117449 usec
  Client: 
    Request count: 153
    Throughput: 67.9898 infer/sec
    Avg client overhead: 0.00%
    Avg send request rate: 8.55 infer/sec
    p50 latency: 117380 usec
    p85 latency: 117469 usec
    p90 latency: 117488 usec
    p95 latency: 117533 usec
    p99 latency: 117599 usec
    Avg gRPC time: 117387 usec (marshal 5 usec + response wait 117381 us

# 1.5 성능 비교

마지막으로, ONNX 런타임을 기준으로 성능을 비교해 보겠습니다. 
* 특히 더 규모가 큰 동시 실행 전반에서 지연 시간이 어떻게 변경되었습니까? 
* 대역폭은 어떻게 변경되었습니까? 관찰된 대역폭 변경 수준을 설명해 주실 수 있습니까? 
* 10개 미만의 동시 실행에서 ONNX 모델이 시간 초과된 이유는 무엇입니까? 이전 동시 실행에서 순수 ONNX 런타임의 지연 시간과 비교할 때 동시 실행 10에서의 TensorRT 지연 시간은 어떻습니까?

강사와 논의하십시오.

<h3 style="color:green;">축하합니다!</h3><br>
TorchScript를 사용해 성공적으로 NLP 모델을 Triton Server에 배포했으며 감소된 정밀도와 TensorRT 최적화를 모두 적용했습니다.
다음 노트북에서는 모델 자체를 최적화하고 효율적인 방식으로 배포하는 방법을 배우게 됩니다. 

다음 노트북을 진행하십시오.<br>
[2.0 모델 호스팅](020_HostingTheModel.ipynb)

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>